In [1]:
# Import python libraries
#
import importlib
import pickle
import os
import torch

# Go one directory back, because all imports are done
# relative to the root of the project.
#
project_root = '..'
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    os.chdir(project_root)

# Imports own modules.
#
import scripts.Visualization as Visualization
import scripts.ModelTrainer as ModelTrainer
import scripts.Utils as utils
import scripts.Simulation_config as Simulation_config
from scripts.Simulation_config import *
import models.Model as Model
import scripts.ModelAdapter as ModelAdapter


In [4]:
importlib.reload(Model)
importlib.reload(ModelTrainer)
importlib.reload(Simulation_config)
importlib.reload(utils)

# Run the whole simulation
#
train_all_models = False
if train_all_models:
    configs = Simulation_config.configs
    ModelTrainer.ModelTrainer().run(configs)

# Print the sumarized simulation results
#
utils.Evaluate_Models.print_results('scripts/outputs/all_train_histories.pkl')


defaultdict(<function scripts.Utils.Evaluate_Models.print_results.<locals>.<lambda>()>,
            {Config_of_one_run(modelSize='5k', doPretraining=True, doTransferLearning=True, aggregation_Count='data/london_loadprofiles_50households_each.pkl', nrOfComunities=20, trainingHistory=365, testSize=92, trainingFuture=0, devSize=58, usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction', 'xLSTM', 'LSTM', 'Transformer_Encoder_Only'), epochs=100): defaultdict(<function scripts.Utils.Evaluate_Models.print_results.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'SyntheticLoadProfile': [21.72661994286418,
                           22.625958118362274,
                           23.135663600869105,
                           20.29830888967362,
                           21.251464932662554,
                           21.867396322971082,
                           19.37890778252043,
                           24.716417482065864,
                           19.891284128

In [5]:
# Plot the losses over the training.
# (To see, if the models are trained stable.)
#
importlib.reload(utils)

plot_only_single_config = True
plotted_config =     Config_of_one_run(ModelSize._5k, DoPretraining.NO, DoTransferLearning.NO, Aggregation_Count._50_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._4_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT)

utils.Evaluate_Models.plot_training_losses_over_epochs('scripts/outputs/all_train_histories.pkl',
                                                       plot_only_single_config,
                                                       plotted_config
                                                       )


Plotted Config:
("Config_of_one_run(modelSize='5k', doPretraining=False, "
 'doTransferLearning=False, '
 "aggregation_Count='data/london_loadprofiles_50households_each.pkl', "
 'nrOfComunities=20, trainingHistory=122, testSize=92, trainingFuture=0, '
 "devSize=58, usedModels=('SyntheticLoadProfile', 'KNN', "
 "'PersistencePrediction', 'xLSTM', 'LSTM', 'Transformer_Encoder_Only'), "
 'epochs=100)')


FileNotFoundError: [Errno 2] No such file or directory: 'scripts/outputs/figs/loss_over_epochs.pdf'

In [ ]:
# Evaluate specific model of given power profiles and given configurations
#

importlib.reload(Visualization)
importlib.reload(Model)
importlib.reload(utils)
importlib.reload(ModelAdapter)

# Define a specific configuration
#
model_type = 'Transformer'
community_id = 0  # chose one of many energy communites
myConfig = Config_of_one_run(ModelSize._5k, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._50_HOUSEHOLDS, NrOfComunities._20, 
            TrainingHistory._12_MONTH, TestSize._3_MONTH, TrainingFuture._0_MONTH, DevSize._2_MONTH, UsedModels.ALL, Epochs.DEFAULT)

modelTrainer = ModelTrainer.ModelTrainer()
loadProfiles, weatherData, public_holidays_timestamps = modelTrainer.load_data(myConfig)
modelAdapter = ModelAdapter.ModelAdapter(public_holidays_timestamps, 
                                            trainHistory = myConfig.trainingHistory,
                                            testSize = myConfig.testSize, 
                                            trainFuture = myConfig.trainingFuture, 
                                            devSize = myConfig.devSize, 
                                            )
X, Y = modelAdapter.transformData(loadProfiles[community_id], weatherData)
path_to_trained_parameters = 'scripts/outputs/all_trained_models.pth'
num_of_features = X['train'].shape[2]
P_el_predicted = modelAdapter.deNormalizeY(Y['test']).flatten().unsqueeze(0)
test_profile = f"scripts/outputs/file_{community_id}.pkl"
my_Model = utils.Deserialize.get_trained_model(path_to_trained_parameters, model_type, test_profile, 
                                                myConfig, num_of_features, modelAdapter)

# Plot the chosen model
#
plotlyApp = Visualization.PlotlyApp(X, Y, my_Model, modelAdapter, None, 'UTC')
plotlyApp.run(myport=8051)


In [2]:
# Print the model parameter sizes
#

importlib.reload(Model)

model_types = ['xLSTM', 'LSTM', 'Transformer_Encoder_Only', 'Transformer_Full']
for model_type in model_types:
    for model_size in ['1k', '2k', '5k', '10k', '20k', '40k', '80k']:

        # Test, if the model is run-able
        num_of_features = 20
        m = Model.Model(model_type, model_size, num_of_features)
        x = torch.zeros((7, 24, num_of_features))
        m.my_model(x)

        # Print the model's parameter count
        nr_of_parameters = m.get_nr_of_parameters(do_print=False)
        print(f"{model_type} - {model_size} has {nr_of_parameters} parameters.")


No CUDA runtime is found, using CUDA_HOME='/home/molu/miniconda3/envs/load_forecasting'


xLSTM - 1k has 1759 parameters.
xLSTM - 2k has 2847 parameters.
xLSTM - 5k has 7151 parameters.
xLSTM - 10k has 11127 parameters.
xLSTM - 20k has 19847 parameters.
xLSTM - 40k has 37719 parameters.
xLSTM - 80k has 84487 parameters.
LSTM - 1k has 955 parameters.
LSTM - 2k has 2191 parameters.
LSTM - 5k has 5075 parameters.
LSTM - 10k has 10071 parameters.
LSTM - 20k has 20175 parameters.
LSTM - 40k has 40335 parameters.
LSTM - 80k has 80835 parameters.
Transformer_Encoder_Only - 1k has 1881 parameters.
Transformer_Encoder_Only - 2k has 2759 parameters.
Transformer_Encoder_Only - 5k has 7161 parameters.
Transformer_Encoder_Only - 10k has 11671 parameters.
Transformer_Encoder_Only - 20k has 19871 parameters.
Transformer_Encoder_Only - 40k has 41871 parameters.
Transformer_Encoder_Only - 80k has 81031 parameters.
Transformer_Full - 1k has 901 parameters.
Transformer_Full - 2k has 2141 parameters.
Transformer_Full - 5k has 4009 parameters.
Transformer_Full - 10k has 13221 parameters.
Transf